In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import nltk
#nltk.download()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
comb_jobs = pd.read_csv('/content/drive/MyDrive/Recommender Eng Files/Combined_Jobs_Final.csv')
experience = pd.read_csv('/content/drive/MyDrive/Recommender Eng Files/Experience.csv')
job_views = pd.read_csv('/content/drive/MyDrive/Recommender Eng Files/Job_Views.csv')
pos_int = pd.read_csv('/content/drive/MyDrive/Recommender Eng Files/Positions_Of_Interest.csv')
job_data = pd.read_csv('/content/drive/MyDrive/Recommender Eng Files/job_data.csv')

In [ ]:
comb_jobs.head(3)

,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,Address,Latitude,Longitude,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
0,111,1,open,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,California,CA,NaN,37.443346,-122.161170,Food and Beverages,Tacolicious' first Palo Alto store just opened...,NaN,8.0,NaN,NaN,Part-Time,NaN,2013-03-12 02:08:28 UTC,2014-08-16 15:35:36 UTC
1,113,1,open,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,California,CA,NaN,37.789830,-122.404268,Food and Beverages,\r\n\r\nNew French Brasserie in S.F. Financia...,NaN,0.0,NaN,NaN,Part-Time,NaN,2013-04-12 08:36:36 UTC,2014-08-16 15:35:36 UTC
2,117,1,open,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,California,CA,NaN,37.795597,-122.402963,Food and Beverages,We are a popular Mediterranean wine bar and re...,NaN,11.0,NaN,NaN,Part-Time,NaN,2013-07-16 09:34:10 UTC,2014-08-16 15:35:37 UTC


In [ ]:
experience.head(3)

,Applicant.ID,Position.Name,Employer.Name,City,State.Name,State.Code,Start.Date,End.Date,Job.Description,Salary,Can.Contact.Employer,Created.At,Updated.At
0,10001,Account Manager / Sales Administration / Quali...,Barcode Resourcing,Bellingham,Washington,WA,2012-10-15,NaN,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
1,10001,Electronics Technician / Item Master Controller,Ryzex Group,Bellingham,Washington,WA,2001-12-01,2012-04-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
2,10001,Machine Operator,comptec inc,Custer,Washington,WA,1997-01-01,1999-01-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC


In [ ]:
job_data.head(3)

,Unnamed: 0,Job.ID,text
0,0,111,server tacolici palo alto part time tacolici f...
1,1,113,kitchen staff chef claud lane san francisco pa...
2,2,117,bartend machka restaur corp. san francisco par...


In [ ]:
pos_int.head(3)

,Applicant.ID,Position.Of.Interest,Created.At,Updated.At
0,10003,security officer,2014-12-12 21:20:54 UTC,2014-12-12 21:20:54 UTC
1,10007,Server,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
2,10007,Bartender,2014-08-14 15:56:44 UTC,2015-02-19 23:21:28 UTC


In [ ]:
job_data.head(3)

,Unnamed: 0,Job.ID,text
0,0,111,server tacolici palo alto part time tacolici f...
1,1,113,kitchen staff chef claud lane san francisco pa...
2,2,117,bartend machka restaur corp. san francisco par...


### **Exploratory Data Analysis**

In [ ]:
comb_jobs.isna().sum()

Job.ID                    0
Provider                  0
Status                    0
Slug                      0
Title                     0
Position                  0
Company                2271
City                    135
State.Name              171
State.Code              171
Address               84054
Latitude                  0
Longitude                 0
Industry              83823
Job.Description          56
Requirements          84090
Salary                83861
Listing.Start           683
Listing.End             167
Employment.Type          10
Education.Required      267
Created.At                0
Updated.At                0
dtype: int64

There are a lot of missing values so I shall just selecct the columns for the jobs corpus. There are 23 columns, however for this dataframe I shall only use the Job.ID, Title, Position, Company, City and Job_Description, after which I shall preprocess.
For the preprocessing step, I  shall
- impute the missing values if any.
- remove stop words.
- remove not alphanumeric characters.
- lemmatize the columns.
-  merge all the columns in order to create a corpus of text for each job.

In [ ]:
comb_jobs_df = comb_jobs[['Job.ID', 'Title', 'Position', 'Company', 'City', 'Employment.Type', 'Job.Description']]
comb_jobs_df.head()

,Job.ID,Title,Position,Company,City,Employment.Type,Job.Description
0,111,Server @ Tacolicious,Server,Tacolicious,Palo Alto,Part-Time,Tacolicious' first Palo Alto store just opened...
1,113,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,Part-Time,\r\n\r\nNew French Brasserie in S.F. Financia...
2,117,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,Part-Time,We are a popular Mediterranean wine bar and re...
3,121,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,Part-Time,● Serve food/drinks to customers in a profess...
4,127,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,Part-Time,"Located at the heart of Hollywood, we are one ..."


In [ ]:
comb_jobs_df.isna().sum()

Job.ID                0
Title                 0
Position              0
Company            2271
City                135
Employment.Type      10
Job.Description      56
dtype: int64

In [ ]:
print(comb_jobs_df[pd.isnull(comb_jobs_df['City'])].shape)
df1 = comb_jobs_df[pd.isnull(comb_jobs_df['City'])]
print(df1[df1['City'] != np.nan]['Company'].unique())
df1.head()

(135, 7)
['St. Francis Hospital' 'CHI Payment Systems' 'Genesis Health Systems'
 'Driveline Retail' 'Volvo Group' 'Home Instead Senior Care'
 'Genesis Health System' 'Academic Year In America'
 'Educational Testing Services' 'CBS Healthcare Services and Staffing']


,Job.ID,Title,Position,Company,City,Employment.Type,Job.Description
3425,142044,Registered Nurse (RN) @ St. Francis Hospital,Registered Nurse (RN),St. Francis Hospital,NaN,Full-Time/Part-Time,Under the direction of the Clinical Coordinato...
3433,142054,Sales Representative - Business Development Op...,Sales Representative - Business Development Op...,CHI Payment Systems,NaN,Full-Time/Part-Time,"If you&rsquo;re energetic, motivated, hardwork..."
3434,142055,New Business Executive @ CHI Payment Systems,New Business Executive,CHI Payment Systems,NaN,Full-Time/Part-Time,"If you&rsquo;re energetic, motivated, hardwork..."
3435,142056,Outside Sales Representative (Business Develop...,Outside Sales Representative (Business Develop...,CHI Payment Systems,NaN,Full-Time/Part-Time,"If you&rsquo;re energetic, motivated, hardwork..."
3436,142057,Outside Sales Representative @ CHI Payment Sys...,Outside Sales Representative,CHI Payment Systems,NaN,Full-Time/Part-Time,"If you&rsquo;re energetic, motivated, hardwork..."


There are 9 companies where the value for which the cities are missing. I did a little research on Google and imputed these values.

In [ ]:
comb_jobs_df.loc[comb_jobs_df.Company == 'CHI Payment Systems', 'City'] = 'Williamsport'
comb_jobs_df.loc[comb_jobs_df.Company == 'Academic Year In America', 'City'] = 'Stamford'
comb_jobs_df.loc[comb_jobs_df.Company == 'CBS Healthcare Services and Staffing ', 'City'] = 'Urbandale'
comb_jobs_df.loc[comb_jobs_df.Company == 'Driveline Retail', 'City'] = 'Coppell'
comb_jobs_df.loc[comb_jobs_df.Company == 'Educational Testing Services', 'City'] = 'Jersey City'
comb_jobs_df.loc[comb_jobs_df.Company == 'Genesis Health System', 'City'] = 'Davenport'
comb_jobs_df.loc[comb_jobs_df.Company == 'Home Instead Senior Care', 'City'] = 'New Albany'
comb_jobs_df.loc[comb_jobs_df.Company == 'St. Francis Hospital', 'City'] = 'Litchfield'
comb_jobs_df.loc[comb_jobs_df.Company == 'Volvo Group', 'City'] = 'Greensboro'
comb_jobs_df.loc[comb_jobs_df.Company == 'CBS Healthcare Services and Staffing', 'City'] = 'Urbandale'
comb_jobs_df['Company'] = comb_jobs_df['Company'].replace(['Genesis Health Systems'], 'Genesis Health System')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
comb_jobs_df.isna().sum()

Job.ID                0
Title                 0
Position              0
Company            2271
City                  2
Employment.Type      10
Job.Description      56
dtype: int64

In [ ]:
df2 = comb_jobs_df[pd.isnull(comb_jobs_df['Employment.Type'])]
comb_jobs_df = comb_jobs_df.dropna(subset = ['Employment.Type'], axis = 0)

In [ ]:
comb_jobs_df['Employment.Type'].unique()

array(['Part-Time', 'Full-Time/Part-Time', 'Seasonal/Temp', 'Per Diem',
       'Intern', 'Full-Time', 'Contract', 'Temporary/seasonal'],
      dtype=object)

In [ ]:
#replacing na values with part time/full time
df2['Employment.Type'] = df2['Employment.Type'].fillna('Full-Time/Part-Time')
comb_jobs_df.groupby(['Employment.Type'])['Company'].count()
df2

,Job.ID,Title,Position,Company,City,Employment.Type,Job.Description
10768,153197,Driving Partner @ Uber,Driving Partner,Uber,San Francisco,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10769,153198,Driving Partner @ Uber,Driving Partner,Uber,Los Angeles,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10770,153199,Driving Partner @ Uber,Driving Partner,Uber,Chicago,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10771,153200,Driving Partner @ Uber,Driving Partner,Uber,Boston,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10772,153201,Driving Partner @ Uber,Driving Partner,Uber,Ann Arbor,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10773,153202,Driving Partner @ Uber,Driving Partner,Uber,Oklahoma,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10774,153203,Driving Partner @ Uber,Driving Partner,Uber,Omaha,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10775,153204,Driving Partner @ Uber,Driving Partner,Uber,Lincoln,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10776,153205,Driving Partner @ Uber,Driving Partner,Uber,Minneapolis,Full-Time/Part-Time,Uber is changing the way the world moves. From...
10777,153206,Driving Partner @ Uber,Driving Partner,Uber,St. Paul,Full-Time/Part-Time,Uber is changing the way the world moves. From...


In [ ]:
comb_jobs_df = pd.concat([comb_jobs_df.iloc[:10768], df2, comb_jobs_df.iloc[10768:]], axis = 0).reset_index(drop = True)

In [ ]:
comb_jobs_df.isna().sum()

Job.ID                0
Title                 0
Position              0
Company            2271
City                  2
Employment.Type       0
Job.Description      56
dtype: int64

##### **Creating the Jobs corpus**

To create the jobs corpora, I shall add the Position, Company, City, Employment.Type and Position columns


In [ ]:
comb_jobs_df['Text'] = comb_jobs_df['Position'].map(str) + ' ' + comb_jobs_df['Company'] + ' ' + comb_jobs_df['City'] + ' ' + comb_jobs_df['Employment.Type'] + ' ' + comb_jobs_df['Job.Description'] + ' ' + comb_jobs_df['Title']
comb_jobs_df.head(2)

,Job.ID,Title,Position,Company,City,Employment.Type,Job.Description,Text
0,111,Server @ Tacolicious,Server,Tacolicious,Palo Alto,Part-Time,Tacolicious' first Palo Alto store just opened...,Server Tacolicious Palo Alto Part-Time Tacolic...
1,113,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,Part-Time,\r\n\r\nNew French Brasserie in S.F. Financia...,Kitchen Staff/Chef Claude Lane San Francisco P...


In [ ]:
comb_jobs_all = comb_jobs_df[['Job.ID', 'Text', 'Title']]
comb_jobs_all = comb_jobs_all.fillna(' ')
comb_jobs_all.head()

,Job.ID,Text,Title
0,111,Server Tacolicious Palo Alto Part-Time Tacolic...,Server @ Tacolicious
1,113,Kitchen Staff/Chef Claude Lane San Francisco P...,Kitchen Staff/Chef @ Claude Lane
2,117,Bartender Machka Restaurants Corp. San Francis...,Bartender @ Machka Restaurants Corp.
3,121,Server Teriyaki House Brisbane Part-Time ● Se...,Server @ Teriyaki House
4,127,Kitchen Staff/Chef Rosa Mexicano - Sunset Los ...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset


In [ ]:
comb_jobs_all.shape

(84090, 3)

In [ ]:
stopword = stopwords.words('english')
stopword_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [ ]:
# Create word tokens
def token_txt(token):
    return token not in stopword_ and token not in list(string.punctuation) and len(token) > 2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "", text)
  text = re.sub("(\\d|\\W)+", " ", text) 
  text = text.replace("nbsp", "")
  clean_text = [wn.lemmatize(word, pos = "v") for word in word_tokenize(text.lower()) if token_txt(word)]
  clean_text2 = [word for word in clean_text if token_txt(word)]
  return " ".join(clean_text2)

In [ ]:
comb_jobs_all['Text'] = comb_jobs_all['Text'].apply(clean_txt) 

In [ ]:
comb_jobs_all.tail()

,Job.ID,Text,Title
84085,82,book keeper national japanese american histori...,Book Keeper @ National Japanese American Histo...
84086,83,kitchen staff chef emporio rulli larkspur part...,Kitchen Staff/Chef @ Emporio Rulli
84087,84,driver onigilly san francisco part time onigil...,Driver @ Onigilly
84088,88,line cook machka restaurants corp san francisc...,Line Cook @ Machka Restaurants Corp.
84089,92,cashier kazoo restaurant san jose part time lo...,Cashier @ Kazoo Restaurant


#### **Term Ferquency - Inverse Document Frequency**

In [ ]:
tfidf_vect = TfidfVectorizer()

# Fitting and transforming the vector
tfidf_comb = tfidf_vect.fit_transform((comb_jobs_all['Text'])) 
tfidf_comb

<84090x50753 sparse matrix of type '<class 'numpy.float64'>'
	with 8264225 stored elements in Compressed Sparse Row format>

#### **Creating the User Corpus**

In [ ]:
job_views.head(3)

,Applicant.ID,Job.ID,Title,Position,Company,City,State.Name,State.Code,Industry,View.Start,View.End,View.Duration,Created.At,Updated.At
0,10000,73666,Cashiers & Valets Needed! @ WallyPark,Cashiers & Valets Needed!,WallyPark,Newark,New Jersey,NJ,NaN,2014-12-12 20:12:35 UTC,2014-12-12 20:31:24 UTC,1129.0,2014-12-12 20:12:35 UTC,2014-12-12 20:12:35 UTC
1,10000,96655,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's,Garden City,New York,NY,NaN,2014-12-12 20:08:50 UTC,2014-12-12 20:10:15 UTC,84.0,2014-12-12 20:08:50 UTC,2014-12-12 20:08:50 UTC
2,10001,84141,Part Time Showroom Sales / Cashier @ Grizzly I...,Part Time Showroom Sales / Cashier,Grizzly Industrial Inc.,Bellingham,Washington,WA,NaN,2014-12-12 20:12:32 UTC,2014-12-12 20:17:18 UTC,286.0,2014-12-12 20:12:32 UTC,2014-12-12 20:12:32 UTC


To create the user corpus, we shall use the Applicant.ID, Job.ID, Position, Company, City columns

In [ ]:
job_view_df = job_views[['Applicant.ID', 'Job.ID', 'Position', 'Company', 'City']]
job_view_df['job_view_text'] = job_view_df['Position'].map(str) + "  " + job_view_df['Company'] + "  " + job_view_df['City']
job_view_df['job_view_text'] = job_view_df['job_view_text'].map(str).apply(clean_txt)
job_view_df['job_view_text'] = job_view_df['job_view_text'].str.lower()
job_view_df = job_view_df[['Applicant.ID', 'job_view_text']]
job_view_df.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,Applicant.ID,job_view_text
0,10000,cashier valet need wallypark newark
1,10000,macys seasonal retail fragrance cashier garden...
2,10001,part time showroom sales cashier grizzly indus...
3,10002,event specialist part time advantage sales mar...
4,10002,bonefish kitchen staff bonefish grill greenville


#### Cleaning the Experience data

In [ ]:
experience.head()

,Applicant.ID,Position.Name,Employer.Name,City,State.Name,State.Code,Start.Date,End.Date,Job.Description,Salary,Can.Contact.Employer,Created.At,Updated.At
0,10001,Account Manager / Sales Administration / Quali...,Barcode Resourcing,Bellingham,Washington,WA,2012-10-15,NaN,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
1,10001,Electronics Technician / Item Master Controller,Ryzex Group,Bellingham,Washington,WA,2001-12-01,2012-04-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
2,10001,Machine Operator,comptec inc,Custer,Washington,WA,1997-01-01,1999-01-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
3,10003,maintenance technician,Winn residental,washington,District of Columbia,DC,NaN,NaN,"Necessary maintenance for ""Make Ready"" Plumbin...",10.0,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC
4,10003,Electrical Helper,michael and son services,alexandria,Virginia,VA,NaN,NaN,repair and services of electrical construction,NaN,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC


In this dataset, only the position name will be taken. 

In [ ]:
# Taking only Position
experience = experience[['Applicant.ID', 'Position.Name']] 

# Cleaning the text
experience['Position.Name'] = experience['Position.Name'].map(str).apply(clean_txt)
experience.head()

,Applicant.ID,Position.Name
0,10001,account manager sales administration quality a...
1,10001,electronics technician item master controller
2,10001,machine operator
3,10003,maintenance technician
4,10003,electrical helper


In [ ]:
experience =  experience.sort_values(by = 'Applicant.ID')
experience = experience.fillna(" ")
experience.head(10)

,Applicant.ID,Position.Name
2763,2,volunteer
2762,2,writer uloop blog
3759,3,market intern
3758,3,server
3757,3,prep cook
6277,6,project assistant
7490,8,deli clerk server cashier food prep order taker
368,11,cashier
809,12,server
810,12,rec leader


One applicant has three positions. I shall merge these positions.

In [ ]:
#adding same rows to a single row
experience = experience.groupby('Applicant.ID', sort = False)['Position.Name'].apply(' '.join).reset_index()
experience.tail(10)

,Applicant.ID,Position.Name
3780,14626,fsr associate multifamily mortgage business as...
3781,14627,cook
3782,14630,product strategist product developer webdesign...
3783,14633,lead fraud analyst account manager development...
3784,14635,lpn lpn cashier coffee maker
3785,14637,nan
3786,14638,doorman bartender facilities department bartender
3787,14639,senate intern student assistant property manag...
3788,14642,program manager business industry train divisi...
3789,14643,inflight wifi market tech support manager foun...


#### **Cleaning Positions of Interest data**

In [ ]:
pos_int_df = pos_int.sort_values(by = 'Applicant.ID')
pos_int_df.head()

,Applicant.ID,Position.Of.Interest,Created.At,Updated.At
6437,96,Server,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
1156,153,Barista,2014-08-14 15:56:43 UTC,2015-02-18 02:35:06 UTC
1155,153,Host,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
1154,153,Server,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
1158,153,Sales Rep,2014-08-14 15:56:47 UTC,2015-03-02 02:13:08 UTC


In [ ]:
pos_int_df = pos_int_df.drop(columns = ['Created.At', 'Updated.At'], axis = 1)

# Cleaning the text
pos_int_df['Position.Of.Interest'] = pos_int_df['Position.Of.Interest'].map(str).apply(clean_txt)
pos_int_df = pos_int_df.fillna(" ")
pos_int_df.head(10)

,Applicant.ID,Position.Of.Interest
6437,96,server
1156,153,barista
1155,153,host
1154,153,server
1158,153,sales rep
1157,153,customer service rep
1952,256,host
1957,256,production area
1956,256,sales rep
1955,256,customer service rep


In [ ]:
pos_int_df = pos_int_df.groupby('Applicant.ID', sort = True)['Position.Of.Interest'].apply(' '.join).reset_index()
pos_int_df.head()

,Applicant.ID,Position.Of.Interest
0,96,server
1,153,barista host server sales rep customer service...
2,256,host production area sales rep customer servic...
3,438,customer service rep barista host server
4,568,receptionist customer service rep book keeper


##### **Creating the final user dataset by merging the Job views, Experience and Positions of Interest**

In [ ]:
jobs_exp_df = job_view_df.merge(experience, left_on = 'Applicant.ID', right_on = 'Applicant.ID', how = 'outer')
jobs_exp_df = jobs_exp_df.fillna(' ')
jobs_exp_df = jobs_exp_df.sort_values(by = 'Applicant.ID')
jobs_exp_df.head()

,Applicant.ID,job_view_text,Position.Name
12370,2,,volunteer writer uloop blog
12371,3,,market intern server prep cook
12372,6,,project assistant
12373,8,,deli clerk server cashier food prep order taker
12374,11,,cashier


In [ ]:
jobs_exp_pos_df = jobs_exp_df.merge(pos_int_df, left_on = 'Applicant.ID', right_on = 'Applicant.ID', how = 'outer')
jobs_exp_pos_df = jobs_exp_pos_df.fillna(' ')
jobs_exp_pos_df = jobs_exp_pos_df.sort_values(by = 'Applicant.ID')
jobs_exp_pos_df.head()

,Applicant.ID,job_view_text,Position.Name,Position.Of.Interest
0,2,,volunteer writer uloop blog,
1,3,,market intern server prep cook,
2,6,,project assistant,
3,8,,deli clerk server cashier food prep order taker,
4,11,,cashier,


In [ ]:
jobs_exp_pos_df['Text'] = jobs_exp_pos_df['job_view_text'].map(str) + jobs_exp_pos_df['Position.Name'] + " " + jobs_exp_pos_df['Position.Of.Interest']
jobs_exp_pos_df.head()


,Applicant.ID,job_view_text,Position.Name,Position.Of.Interest,Text
0,2,,volunteer writer uloop blog,,volunteer writer uloop blog
1,3,,market intern server prep cook,,market intern server prep cook
2,6,,project assistant,,project assistant
3,8,,deli clerk server cashier food prep order taker,,deli clerk server cashier food prep order tak...
4,11,,cashier,,cashier


In [ ]:
user_final_df = jobs_exp_pos_df[['Applicant.ID', 'Text']]
user_final_df.head()

,Applicant.ID,Text
0,2,volunteer writer uloop blog
1,3,market intern server prep cook
2,6,project assistant
3,8,deli clerk server cashier food prep order tak...
4,11,cashier


In [ ]:
user_final_df['Text'] = user_final_df['Text'].apply(clean_txt)
user_final_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Applicant.ID,Text
0,2,volunteer writer uloop blog
1,3,market intern server prep cook
2,6,project assistant
3,8,deli clerk server cashier food prep order taker
4,11,cashier


In [ ]:
user = 126
index1 = np.where(user_final_df['Applicant.ID'] == user1)[0][0]
user_seghe = user_final_df.iloc[[index1]]
user_seghe

,Applicant.ID,Text
72,126,server


In [ ]:
user = 719
index2 = np.where(user_final_df['Applicant.ID'] == user2)[0][0]
user_gbade = user_final_df.iloc[[index2]]
user_gbade

,Applicant.ID,Text
372,719,chaplain


#### **Building the Recommender Systems**

##### **Computing the Cosine Similarity using TF-IDF**

In [ ]:
user_tfidf_seghe = tfidf_vect.transform(user_seghe['Text'])
cos_sim_tfidf_seghe = map(lambda x: cosine_similarity(user_tfidf_seghe, x), tfidf_comb)

In [ ]:
user_tfidf_gbade = tfidf_vect.transform(user_gbade['Text'])
cos_sim_tfidf_gbade = map(lambda x: cosine_similarity(user_tfidf_gbade, x), tfidf_comb)

In [ ]:
rec1 = list(cos_sim_tfidf_seghe)
rec2 = list(cos_sim_tfidf_gbade)

##### **Computing the Top-N Recommendation by score**

In [ ]:
def get_recommendation(top, comb_jobs_all, scores):
  recommendation = pd.DataFrame(columns = ['Applicant_ID', 'Job_ID',  'Title', 'Score'])
  count = 0
  for i in top:
      recommendation.at[count, 'Applicant_ID'] = user
      recommendation.at[count, 'Job_ID'] = comb_jobs_all['Job.ID'][i]
      recommendation.at[count, 'Title'] = comb_jobs_all['Title'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

##### **Top Recommendations with TF-IDF**

In [ ]:
top10_seghe_tfidf = sorted(range(len(rec1)), key = lambda i: rec1[i], reverse = True)[:10]
list_scores_seghe_tfidf = [rec1[i][0][0] for i in top10_seghe_tfidf]
get_recommendation(top10_seghe_tfidf, comb_jobs_all, list_scores_seghe_tfidf)

,Applicant_ID,Job_ID,Title,Score
0,126,290852,Server @ Presbyterian Senior Living,0.579385
1,126,277156,Jr. Server Administrator (2) @ American Cybers...,0.562764
2,126,290980,Marketplace Server @ Presbyterian Senior Living,0.534239
3,126,271185,Server @ Vi,0.533653
4,126,277114,Server @ Vi,0.533653
5,126,271184,Server Assistant @ Vi,0.50932
6,126,257044,Marketing Assistant/ Server/ Host - To $11/hr ...,0.430508
7,126,145399,Banquet Server @ First Class Workforce Solutions,0.421348
8,126,303620,Server @ Benchmark Senior Living,0.417929
9,126,278034,Part-time or Full-time AM & PM Fine Dining Ser...,0.390875


In [ ]:
top10_gbade_tfidf = sorted(range(len(rec2)), key = lambda i: rec2[i], reverse = True)[:10]
list_scores_gbade_tfidf = [rec2[i][0][0] for i in top10_gbade_tfidf]
get_recommendation(top10_gbade_tfidf, comb_jobs_all, list_scores_gbade_tfidf)

,Applicant_ID,Job_ID,Title,Score
0,719,265426,Part-Time Chaplain @ Tyson Foods Incorporated,0.753448
1,719,244551,Part-Time Chaplain @ Tyson Foods Incorporated,0.735256
2,719,172882,Part-Time Chaplain @ Tyson Foods Incorporated,0.730662
3,719,250395,Part-Time Chaplain @ Tyson Foods Incorporated,0.730622
4,719,252026,Part-Time Chaplain @ Tyson Foods Incorporated,0.729452
5,719,276623,Part-Time Chaplain @ Tyson Foods Incorporated,0.710595
6,719,313190,Part-Time Chaplain @ Tyson Foods Incorporated,0.704596
7,719,263828,Chaplain On Call - California @ VITAS Healthcare,0.701806
8,719,148052,Chaplain @ The Evangelical Lutheran Good Samar...,0.677102
9,719,279991,Chaplain @ Catholic Health Initiatives,0.665359


In [ ]:
count_vect = CountVectorizer()

# Fitting and transforming the vectorizer
count_comb = count_vect.fit_transform((comb_jobs_all['Text'])) #fitting and transforming the vector
count_comb

<84090x50753 sparse matrix of type '<class 'numpy.int64'>'
	with 8264225 stored elements in Compressed Sparse Row format>

In [ ]:
user_count_seghe = count_vect.transform(user_seghe['Text'])
cos_sim_count_seghe = map(lambda x: cosine_similarity(user_count_seghe, x), count_comb)

In [ ]:
user_count_gbade = count_vect.transform(user_gbade['Text'])
cos_sim_count_gbade = map(lambda x: cosine_similarity(user_count_gbade, x), count_comb)

In [ ]:
rem1 = list(cos_sim_count_seghe)
rem2 = list(cos_sim_count_gbade)

##### **Top Recommendations with Count Vectorizer**

In [ ]:
top10_seghe_count = sorted(range(len(rem1)), key = lambda i: rem1[i], reverse = True)[:10]
list_scores_seghe_count = [rem1[i][0][0] for i in top10_seghe_count]
get_recommendation(top10_seghe_count, comb_jobs_all, list_scores_seghe_count)

,Applicant_ID,Job_ID,Title,Score
0,126,277156,Jr. Server Administrator (2) @ American Cybers...,0.529999
1,126,290852,Server @ Presbyterian Senior Living,0.516398
2,126,290980,Marketplace Server @ Presbyterian Senior Living,0.492366
3,126,524,Server @ Yuzu,0.447214
4,126,271185,Server @ Vi,0.433861
5,126,277114,Server @ Vi,0.433861
6,126,309,Server @ Piperade,0.428571
7,126,145399,Banquet Server @ First Class Workforce Solutions,0.427482
8,126,146441,Dishwasher/Server @ Claremont Place,0.414781
9,126,251932,Server @ Carrabba's Italian Grill,0.40452


In [ ]:
top10_gbade_count = sorted(range(len(rem2)), key = lambda i: rem2[i], reverse = True)[:10]
list_scores_gbade_count = [rem2[i][0][0] for i in top10_gbade_count]
get_recommendation(top10_gbade_count, comb_jobs_all, list_scores_gbade_count)

,Applicant_ID,Job_ID,Title,Score
0,719,265426,Part-Time Chaplain @ Tyson Foods Incorporated,0.576782
1,719,172882,Part-Time Chaplain @ Tyson Foods Incorporated,0.555294
2,719,250395,Part-Time Chaplain @ Tyson Foods Incorporated,0.555294
3,719,252026,Part-Time Chaplain @ Tyson Foods Incorporated,0.555294
4,719,276623,Part-Time Chaplain @ Tyson Foods Incorporated,0.532952
5,719,313190,Part-Time Chaplain @ Tyson Foods Incorporated,0.531085
6,719,263828,Chaplain On Call - California @ VITAS Healthcare,0.524222
7,719,244551,Part-Time Chaplain @ Tyson Foods Incorporated,0.520579
8,719,313189,Part-Time Chaplain @ Tyson Foods Incorporated,0.501886
9,719,148052,Chaplain @ The Evangelical Lutheran Good Samar...,0.472456


#### **Recommendations Using KNN**

In [ ]:
n_neighbors = 11
KNN = NearestNeighbors(n_neighbors, p = 2)
KNN.fit(tfidf_comb)
knn_seghe = KNN.kneighbors(user_tfidf_seghe, return_distance = True) 
knn_gbade = KNN.kneighbors(user_tfidf_gbade, return_distance = True) 

In [ ]:
knn_seghe[0][0][1:]

array([0.93513244, 0.96515431, 0.96576098, 0.96576098, 0.99063614,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [ ]:
knn_gbade[0][0][1:]

array([0.72765877, 0.73394482, 0.73399983, 0.73559282, 0.76079603,
       0.76864052, 0.77226143, 0.80361442, 0.81809684, 0.81894017])

##### **Top Recommendations using KNN**

In [ ]:
top10_seghe_knn = knn_seghe[1][0][1:]
index_score_seghe = knn_seghe[0][0][1:]

get_recommendation(top10_seghe_knn, comb_jobs_all, index_score_seghe)

,Applicant_ID,Job_ID,Title,Score
0,126,277156,Jr. Server Administrator (2) @ American Cybers...,0.935132
1,126,290980,Marketplace Server @ Presbyterian Senior Living,0.965154
2,126,271185,Server @ Vi,0.965761
3,126,277114,Server @ Vi,0.965761
4,126,271184,Server Assistant @ Vi,0.990636
5,126,253339,"BOOKKEEPING, PAYROLL",1
6,126,253332,Part-Time CSR/Collections Agent,1
7,126,253275,Administrative Assistant,1
8,126,253324,Dental Assistant (Immediate Need),1
9,126,253293,Part time Automotive Photo and Merchandising A...,1


In [ ]:
top10_gbade_knn = knn_gbade[1][0][1:]
index_score_gbade = knn_gbade[0][0][1:]

get_recommendation(top10_gbade_knn, comb_jobs_all, index_score_gbade)

,Applicant_ID,Job_ID,Title,Score
0,719,244551,Part-Time Chaplain @ Tyson Foods Incorporated,0.727659
1,719,172882,Part-Time Chaplain @ Tyson Foods Incorporated,0.733945
2,719,250395,Part-Time Chaplain @ Tyson Foods Incorporated,0.734
3,719,252026,Part-Time Chaplain @ Tyson Foods Incorporated,0.735593
4,719,276623,Part-Time Chaplain @ Tyson Foods Incorporated,0.760796
5,719,313190,Part-Time Chaplain @ Tyson Foods Incorporated,0.768641
6,719,263828,Chaplain On Call - California @ VITAS Healthcare,0.772261
7,719,148052,Chaplain @ The Evangelical Lutheran Good Samar...,0.803614
8,719,279991,Chaplain @ Catholic Health Initiatives,0.818097
9,719,313189,Part-Time Chaplain @ Tyson Foods Incorporated,0.81894
